In [1]:
import os
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Этап 1. Загрузка текстов из файлов

In [2]:
TEXT_FILES_DIR = "garant"

In [3]:
def load_texts_from_folder(folder_path):
    texts = []
    filenames = []
    for file in tqdm(sorted(os.listdir(folder_path)), desc="Загрузка текстов"):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
                texts.append(text)
                filenames.append(file)
    return texts, filenames

In [4]:
texts, filenames = load_texts_from_folder(TEXT_FILES_DIR)

Загрузка текстов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1628/1628 [00:02<00:00, 634.12it/s]


## Этап 2. Создание эмбеддингов

In [5]:
MODEL_NAME = "intfloat/multilingual-e5-large"
model = SentenceTransformer(MODEL_NAME)

In [6]:
text_embeddings = model.encode(texts, batch_size=16, convert_to_numpy=True, show_progress_bar=True)
print(f"\nРазмерность эмбеддингов: {text_embeddings.shape}")

Batches:   0%|          | 0/102 [00:00<?, ?it/s]


Размерность эмбеддингов: (1628, 1024)


## Этап 3: Создание FAISS-индекса

FAISS используется для быстрого поиска среди эмбеддингов

In [7]:
norms = np.linalg.norm(text_embeddings, axis=1, keepdims=True)
text_embeddings_normalized = text_embeddings / norms
dimension = text_embeddings.shape[1]

Создаём FAISS-индекс

In [8]:
index = faiss.IndexFlatL2(dimension)
index.add(text_embeddings_normalized.astype('float32'))

Сохраняем индекс для последующего использования

In [9]:
faiss.write_index(index, "results_Microsoft_Default/text_index.faiss")
np.save("results_Microsoft_Default/filenames.npy", np.array(filenames))  # сохраняем список имён файлов

## Этап 4: Функция поиска похожих текстов

In [10]:
def find_similar_texts(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True) # преобразуем запрос в эмбеддинг
    distances, indices = index.search(query_embedding, top_k) # выполняем поиск в FAISS

    print("Наиболее похожие тексты:")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {filenames[idx]} (дистанция: {distances[0][i]:.4f})")
        print(texts[idx][:300] + "...")  # выводим первые 300 символов текста
        print("-" * 80)

In [11]:
query_text = "Международный день инвалидов в Хабаровске"
find_similar_texts(query_text, top_k=20)

Наиболее похожие тексты:
1. garant_1261.txt (дистанция: 0.2817)
﻿
Постановление Мэра г. Хабаровска
от 11 ноября 2004 г. N 1650
"О проведении Международного дня инвалидов в городе Хабаровске в 2004 году"

В связи с проведением 3 декабря 2004 года Международного дня инвалидов и в целях привлечения внимания широких слоев общественности города, предприятий, учрежден...
--------------------------------------------------------------------------------
2. garant_0291.txt (дистанция: 0.4202)
﻿Зарегистрировано в Минюсте РФ 22 декабря 2004 г.
Регистрационный N 6226

      Приказ Министерства здравоохранения и социального развития РФ
                       от 29 ноября 2004 г. N 287
  "Об утверждении формы индивидуальной программы реабилитации инвалида,
   выдаваемой федеральными учреждени...
--------------------------------------------------------------------------------
3. garant_0522.txt (дистанция: 0.4229)
﻿
Постановление Мэра г. Хабаровска
от 1 декабря 2004 г. N 1762
"О проведении в 2005 году